# Zelf-gedefinieerde datatypes

- bijv. punt (coördinaten)
- bijv. kleuren ("enumeration type")
- bijv. vormen (tekenprogramma?) ("disjoint union")
    - wat zijn de voorbeelden in Dahl, Dijkstra, Hoare?
- alternatieven, bijv. verschillende soorten operatoren en operanden (termen in een expressie)

Verwerking hiervan: case analysis (notatie in Haskell? verkorten notatie bij functies, uitgebreidere notatie in expressies?

- case analysis
- pattern matching
    - nb: dit kan al in de formele parameters van een functie, bijv `f [] = ... ; f (x:xs) = ...`

## Eenvoudige data-types: alternatieve waarden

De data-types die we tot nu toe gezien hebben, zowel de elementaire types als de samengestelde types lijsten en tupels, zijn voor-gedefinieerd. We kunnen ook zelf data-types definiëren.

De eenvoudigste vorm van een type-definitie geeft een *opsomming van de namen* van de verschillende waarden. Deze waarde-namen heten ook wel *constructors* - waarom, zal straks duidelijker worden.

**De namen van types en de namen van constructors beginnen in Haskell met een hoofdletter.**

In [ ]:
data Color = Red | Orange | Yellow | Green | Blue | Indigo | Violet -- deriving (Show)

We kunnen de waarde van een kleur toekennen aan een naam, bijvoorbeeld:

In [ ]:
myColor = Yellow

We kunnen functies definiëren om met deze waarden te "rekenen". Eigenlijk moeten we alle vormen van rekenen met deze waarden nu zelf definiëren. De functies die we zo definiëren moeten voor elke Color-waarde een resultaat definiëren:

nextColor :: Color -> Color
nextColor Red = Orange
nextColor Orange = Yellow
nextColor Yellow = Green
nextColor Green = Blue
nextColor Indigo = Violet
nextColor Violet = Violet

We hebben hier de functie `nextColor` gedefinieerd door een definitie voor elke mogelijke vorm van `Color`. Deze constructie heet ook wel *pattern matching*.

Een alternatief is om een "case analysis" binnen de functie-definitie uit te voeren, zoals in dit voorbeeld:

```Haskell
nextColor :: Color -> Color
nextColor c = 
    case c of
    Red -> Orange
    Orange -> Yellow 
    Yellow -> Green
    Green -> Blue
    Indigo -> Violet
    Violet -> Violet
```


We kunnen niet zomaar het resultaat van een Color-expressie tonen: daarvoor moeten we eerst de functie `Show` voor dit type definiëren:
een expressie van type Color uitrekenen en 

In [ ]:
instance Show Color where
-- show :: Color -> String
    show Red = "Red" -- of: "Rood"
    show Orange = "Orange"
    show Yellow = "Yellow"
    show Green = "Green"
    show Blue = "Blue"
    show Indigo = "Indigo"
    show Violet = "Violet"

In [ ]:
nextColor myColor

Een aantal van de types die voorgedefinieerd zijn, zijn gedefinieerd in de "standard prelude": de standaard-library van Haskell. Het type Bool is daarin bijvoorbeeld gedefinieerd als:

```Haskell
data  Bool  =  False | True
```

In [ ]:
nextColor :: Color -> Color
nextColor c = 
    case c of
    Red -> Orange
    Orange -> Yellow 
    Yellow -> Green
    Green -> Blue
    Indigo -> Violet
    Violet -> Violet

In [ ]:
nextColor Green

## Alternatieven met attributen

In het eenvoudige data-type `Color` spreken de waarden voor zich. Maar bij complexere types kunnen de waarden *attributen* hebben.

Een voorbeeld hiervan is het type `Shape` (voor een 2-dimensionale geometrische vorm). We onderscheiden (in eerste instantie) cirkels en rechthoeken:

* een cirkel wordt bepaald door het middelpunt (coördinaten) en de straal (Float)
* een rechthoek wordt bepaald door de coördinaten van de linker-bovenhoek (punt) en de rechter-onderhoek (punt).
* een punt is een *2-tupel* van Floats: x- en y-coördinaten.

Andere mogelijke elementen zijn:

* lijnstuk (met 2 coördinaten: begin- en eindpunt)
* pad (een lijst van coördinaten; reeks aaneengesloten lijnstukken)

(NB: voor een rechthoek die ook "scheef kan staan" hebben we meer gegevens nodig: tenminste 3 punten? Of naast de basis-punten ook een rotatie? Voor een cirkel hebben we geen rotatie nodig... Een alternatief is dat we later een vorm introduceren die het assenstelsel van de omvatte figuren aanpast, met schaling, rotatie, en translatie.)

(Een alternatief voor de rechhoek is: positie, afmetingen (lengte, hoogte), en eventueel hoek. Uit die ene coördinaat kun je dan de ccoördinaten van de andere punten uitrekenen.)

(SVG: een rechthoek heeft een positie en afmetingen: *width* en *height*; en veel andere attributen, zoals kleur, lijndikte, lijnkleur, enz. En: *transform* is gekoppeld aan een *group*.)

**Vraag*: attributen zoals lijndikte, lijnkleur en vulkeur zijn eigenlijk gemeenschappelijk voor alle vormen. Hoe kun je dat het best uitdrukken in Haskell? Een vorm van "overerving"?

Mogelijke functies voor deze vormen:

* translatie (coördinaten aanpassen)
* schaling (straal en afmetingen aanpassen; voor rechthoek, alleen tweede coördinaat)
* berekenen van de oppervlakte.

Als afkorting voeren we het type `Point` in: een tupel 2 getallen: de x- en y-coördinaat van het punt.

In [1]:
type Point = (Float, Float)

Twee eenvoudige vormen: een cirkel, met middelpunt en straal; en een rechthoek, met positie, breedte en hoogte.

*(Hebben we geen andere manier om deze eigenschappen te benoemen en te documenteren?)*

In [2]:
data Shape = Circle Point Float | Rect Point Float Float

We kunnen nu een functie definiëren die de oppervlakte van een (gesloten) figuur uitrekent. We moeten in die functie de verschillende vormen onderscheiden. In Haskell kan dat door de functie voor elk alternatief afzonderlijk te definiëren:

In [3]:
area :: Shape -> Float
area (Circle centre radius) = pi * radius * radius
area (Rect position width height) = width * height

In [19]:
shape1 = Circle (50, 50) 20
shape2 = Rect (20, 30) 10 20

In [11]:
area shape1

314.15927


In [12]:
area shape2

200.0


Voorbeelden:

- `translate :: Point -> Shape -> Shape`
- `scale :: Float -> Shape -> Shape`
- `tosvg :: Shape -> String`

In [16]:
tosvg :: Shape -> String
tosvg (Circle (mx, my) r) = "<circle cx=" ++ (show mx) ++ " cy=" ++ (show my) ++ " r=" ++ (show r) ++ " /> \n"
tosvg (Rect (mx, my) w h) = "<rect x=" ++ (show mx) ++ " y=" ++ (show my) ++ " w=" ++ (show w) ++ " h=" ++ (show h) ++ " />"

In [17]:
tosvg shape1

"<circle cx=50.0 cy=50.0 r=10.0 /> \n"


Om dit te kunnen gebruiken in een SVG-figuur, moeten we deze string-waarde als uitvoer hebben, in plaats van in de Haskell-notatie; met andere woorden, zonder de quote-tekens, en met de escape-tekens zooals `'\n'` verwerkt. Hiervoor gebruiken we de functie `putStr`:

In [18]:
putStr (tosvg shape1)

<circle cx=50.0 cy=50.0 r=10.0 /> 


In [18]:
putStr (tosvg shape1)

<circle cx=50.0 cy=50.0 r=10.0 /> 


De inhoud van deze cel kopiëren we naar de cel met SVG-invoer, in het svg-display notebook. (zie de handleiding daar verder)